In [2]:
!git clone https://github.com/JaidedAI/EasyOCR.git && cd EasyOCR

Cloning into 'EasyOCR'...
remote: Enumerating objects: 2750, done.
remote: Counting objects: 100% (846/846), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 2750 (delta 780), reused 756 (delta 756), pack-reused 1904 (from 1)
Receiving objects: 100% (2750/2750), 157.82 MiB | 17.85 MiB/s, done.
Resolving deltas: 100% (1689/1689), done.
Updating files: 100% (313/313), done.


In [4]:
%cd /content/EasyOCR/trainer
!pip install pytorch=2.2.0 pandas
!mv test.py test2.py
!sed -i 's/from test import validation/from test2 import validation/g' train.py
!sed -i 's/image, text = data_loader_iter.next()/image, text = next(data_loader_iter)/g' dataset.py
!sed -i 's/image, text = self.dataloader_iter_list[i].next()/image, text = next(self.dataloader_iter_list[i])/g' dataset.py
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

/content/EasyOCR/trainer


ImportError: cannot import name '_accumulate' from 'torch._utils' (/usr/local/lib/python3.11/dist-packages/torch/_utils.py)

In [ ]:
cudnn.benchmark = True
cudnn.deterministic = False

In [ ]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [ ]:
opt = get_config("config_files/en_filtered_config.yaml")
train(opt, amp=False)